# Loading Yelp's Data and Processing

## Import the packages

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame

%matplotlib inline
plt.style.use('default')
print(plt.style.available)
#print all available styles

print (pd.__version__)

import os
from IPython.core.display import HTML

['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', '_classic_test']
0.20.3


# Yelp Dataset Challenge URL

This link gives you an idea of the structure of Data Schema: all JSON files with each line as a distinct record
https://www.yelp.com/dataset/challenge

Except dataset_photo.json, there are 5 tables:

1. Business
2. Review
3. Checkins
4. Tips
5. Users

Further down this notebook, we will see that the Reviews table is a 1GB file that's not easy to load directly into IPython RAM, so we'll chop it up into 4 pieces along the way of our cleaning process.

In [2]:
YELP_DATACHALLENGE_URL = "https://www.yelp.com/dataset/challenge"
tag = '<iframe src = "{0}" width = "1000px" height = "333px">'.format(YELP_DATACHALLENGE_URL)
HTML(tag)

## Read data from file and load to Pandas DataFrame

In [3]:
file_business, file_checkin, file_tip, file_user, file_review = [
    'yelp_academic_dataset_business.json',
    'yelp_academic_dataset_checkin.json',
    'yelp_academic_dataset_tip.json',
    'yelp_academic_dataset_user.json',
    'yelp_academic_dataset_review.json'
]

### Business Data

In [4]:
# Open file
f = open(file_business,encoding="utf8")
first_line = f.readline()
first_line_json = json.loads(first_line)

print (type(first_line))
print (first_line)
print (type(first_line_json))
print (first_line_json)

<class 'str'>
{"business_id":"Apn5Q_b6Nz61Tq4XzPdf9A","name":"Minhas Micro Brewery","neighborhood":"","address":"1314 44 Avenue NE","city":"Calgary","state":"AB","postal_code":"T2E 6L6","latitude":51.0918130155,"longitude":-114.031674872,"stars":4.0,"review_count":24,"is_open":1,"attributes":{"BikeParking":"False","BusinessAcceptsCreditCards":"True","BusinessParking":"{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}","GoodForKids":"True","HasTV":"True","NoiseLevel":"average","OutdoorSeating":"False","RestaurantsAttire":"casual","RestaurantsDelivery":"False","RestaurantsGoodForGroups":"True","RestaurantsPriceRange2":"2","RestaurantsReservations":"True","RestaurantsTakeOut":"True"},"categories":"Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel","hours":{"Monday":"8:30-17:0","Tuesday":"11:0-21:0","Wednesday":"11:0-21:0","Thursday":"11:0-21:0","Friday":"11:0-21:0","Saturday":"11:0-21:0"}}

<class 'dict'>
{'business_id': 'Apn5Q_b6Nz61Tq4XzPdf9A'

In [5]:
with open(file_business,encoding="utf8") as f:
    df_business = pd.DataFrame(json.loads(line) for line in f)

In [6]:
df_business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
3,211 W Monroe St,None,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,None,1,33.449999,-112.076979,Geico Insurance,,85003,8,1.5,AZ
4,2005 Alyth Place SE,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,51.035591,-114.027366,Action Engine,,T2H 0N5,4,2.0,AB


In [7]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188593 entries, 0 to 188592
Data columns (total 15 columns):
address         188593 non-null object
attributes      162807 non-null object
business_id     188593 non-null object
categories      188052 non-null object
city            188593 non-null object
hours           143791 non-null object
is_open         188593 non-null int64
latitude        188587 non-null float64
longitude       188587 non-null float64
name            188593 non-null object
neighborhood    188593 non-null object
postal_code     188593 non-null object
review_count    188593 non-null int64
stars           188593 non-null float64
state           188593 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 21.6+ MB


### Checkin Data

In [8]:
with open(file_checkin,encoding="utf8") as f:
    df_checkin = pd.DataFrame(json.loads(line) for line in f)

In [9]:
df_checkin.head()

,business_id,time
0,7KPBkxAOEtb3QeIL9PEErg,"{'Fri-0': 2, 'Sat-0': 1, 'Sun-0': 1, 'Wed-0': ..."
1,kREVIrSBbtqBhIYkTccQUg,"{'Mon-13': 1, 'Thu-13': 1, 'Sat-16': 1, 'Wed-1..."
2,tJRDll5yqpZwehenzE2cSg,"{'Thu-0': 1, 'Mon-1': 1, 'Mon-12': 1, 'Sat-16'..."
3,tZccfdl6JNw-j5BKnCTIQQ,"{'Sun-14': 1, 'Fri-18': 1, 'Mon-20': 1}"
4,r1p7RAMzCV_6NPF0dNoR3g,"{'Sat-3': 1, 'Sun-18': 1, 'Sat-21': 1, 'Sat-23..."


In [10]:
df_checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157075 entries, 0 to 157074
Data columns (total 2 columns):
business_id    157075 non-null object
time           157075 non-null object
dtypes: object(2)
memory usage: 2.4+ MB


### Tip Data

In [11]:
with open(file_tip,encoding="utf8") as f:
    df_tip = pd.DataFrame(json.loads(line) for line in f)
    
df_tip.head(2)
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1185348 entries, 0 to 1185347
Data columns (total 5 columns):
business_id    1185348 non-null object
date           1185348 non-null object
likes          1185348 non-null int64
text           1185348 non-null object
user_id        1185348 non-null object
dtypes: int64(1), object(4)
memory usage: 45.2+ MB


### User Data

In [12]:
with open(file_user,encoding="utf8") as f:
    df_user = pd.DataFrame(json.loads(line) for line in f)
    
df_user.head(2)
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1518169 entries, 0 to 1518168
Data columns (total 22 columns):
average_stars         1518169 non-null float64
compliment_cool       1518169 non-null int64
compliment_cute       1518169 non-null int64
compliment_funny      1518169 non-null int64
compliment_hot        1518169 non-null int64
compliment_list       1518169 non-null int64
compliment_more       1518169 non-null int64
compliment_note       1518169 non-null int64
compliment_photos     1518169 non-null int64
compliment_plain      1518169 non-null int64
compliment_profile    1518169 non-null int64
compliment_writer     1518169 non-null int64
cool                  1518169 non-null int64
elite                 1518169 non-null object
fans                  1518169 non-null int64
friends               1518169 non-null object
funny                 1518169 non-null int64
name                  1518169 non-null object
review_count          1518169 non-null int64
useful                15181

### Review Data

In [13]:
with open(file_review,encoding="utf8") as f:
    df_review = pd.DataFrame(json.loads(line) for line in f)
    
df_review.head(2)
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5996996 entries, 0 to 5996995
Data columns (total 9 columns):
business_id    object
cool           int64
date           object
funny          int64
review_id      object
stars          int64
text           object
useful         int64
user_id        object
dtypes: int64(4), object(5)
memory usage: 411.8+ MB


## Filter business data by city and category

** Create filters/masks **
* creat filters that selects business
  + that are located in "Las Vegas"
  + that contains "Restaurants" in their category(I may need to filter null categories first)
 
   

In [14]:
# Create pandas DataFrame filters

# city Las Vegas: bool
cond_city = df_business['city'] == "Las Vegas"

# isnull: bool
cond_category_not_null = ~df_business['categories'].isnull()

#restaurant in categories 
#first, apply(str) convert categories to strings; then check if it contains Restaurants
cond_category_restanrant = df_business['categories'].apply(str).str.contains('Restaurants')

In [15]:
# Create filtered DataFrame, and name it df_filtered
df_filtered = df_business[cond_city&cond_category_not_null&cond_category_restanrant]

In [16]:
df_filtered.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
19,"3940 Martin Luther King Blvd, Ste 101","{'BikeParking': 'True', 'BusinessAcceptsCredit...",vJIuDBdu01vCA8y1fwR1OQ,"American (Traditional), Food, Bakeries, Restau...",Las Vegas,"{'Tuesday': '11:0-18:0', 'Wednesday': '11:0-18...",0,36.192284,-115.159272,CakesbyToi,,89106,3,1.5,NV
32,2255 N Rampart Blvd,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...",kgffcoxT6BQp-gJ-UQ7Czw,"Fast Food, Restaurants, Sandwiches",Las Vegas,"{'Monday': '9:0-21:0', 'Tuesday': '9:0-21:0', ...",1,36.201794,-115.281981,Subway,Summerlin,89128,13,2.5,NV
33,2227 N Rampart Blvd,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",0jtRI7hVMpQHpUVtUy4ITw,"Beer, Wine & Spirits, Italian, Food, American ...",Las Vegas,"{'Monday': '7:0-14:30', 'Tuesday': '7:0-19:0',...",1,36.201990,-115.283122,Omelet House Summerlin,Summerlin,89128,242,4.0,NV
61,7930 W Tropical Pkwy,"{'HasTV': 'False', 'RestaurantsReservations': ...",JJEx5wIqs9iGGATOagE8Sg,"Mexican, Restaurants",Las Vegas,None,0,36.271169,-115.267759,Baja Fresh Mexican Grill,Centennial,89149,4,2.0,NV
141,4505 E Bonanza Rd,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...",zhxnD7J5_sCrKSw5cwI9dQ,"Chicken Wings, Restaurants, Fast Food",Las Vegas,"{'Monday': '10:0-23:0', 'Tuesday': '10:0-23:0'...",1,36.173140,-115.077945,Popeyes Louisiana Kitchen,Sunrise,89110,16,1.5,NV


**Keep relevant columns**
* Only keep some useful columns
  * business_id
  * name 
  * categories
  * stars

In [17]:
selected_features= [u'business_id', u'name', u'categories', u'stars']

In [18]:
# Make a DataFrame that contains only the abovementioned columns, and name it as df_selected_business
df_selected_business = df_filtered[selected_features]
df_selected_business.head(2)

,business_id,name,categories,stars
19,vJIuDBdu01vCA8y1fwR1OQ,CakesbyToi,"American (Traditional), Food, Bakeries, Restau...",1.5
32,kgffcoxT6BQp-gJ-UQ7Czw,Subway,"Fast Food, Restaurants, Sandwiches",2.5


In [19]:
# Rename the column name "stars" to "avg_stars" to avoid naming conflicts with review dataset
df_selected_business.rename(index = str, columns={"stars":"avg_stars"}, inplace = True)
df_selected_business.head()

C:\Users\tinag\AnacondaProjects\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,business_id,name,categories,avg_stars
19,vJIuDBdu01vCA8y1fwR1OQ,CakesbyToi,"American (Traditional), Food, Bakeries, Restau...",1.5
32,kgffcoxT6BQp-gJ-UQ7Czw,Subway,"Fast Food, Restaurants, Sandwiches",2.5
33,0jtRI7hVMpQHpUVtUy4ITw,Omelet House Summerlin,"Beer, Wine & Spirits, Italian, Food, American ...",4.0
61,JJEx5wIqs9iGGATOagE8Sg,Baja Fresh Mexican Grill,"Mexican, Restaurants",2.0
141,zhxnD7J5_sCrKSw5cwI9dQ,Popeyes Louisiana Kitchen,"Chicken Wings, Restaurants, Fast Food",1.5


In [20]:
# Check the infomation

df_selected_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6148 entries, 19 to 188567
Data columns (total 4 columns):
business_id    6148 non-null object
name           6148 non-null object
categories     6148 non-null object
avg_stars      6148 non-null float64
dtypes: float64(1), object(3)
memory usage: 240.2+ KB


### Save the result to csv files

In [21]:
# Save to /selected_business.csv for the next steps
df_selected_business.to_csv("select_business.csv", index = False, encoding = 'utf8')

In [22]:
# Try to reload the csv file and check if it works
pd.read_csv("select_business.csv", encoding = 'utf8').head()

,business_id,name,categories,avg_stars
0,vJIuDBdu01vCA8y1fwR1OQ,CakesbyToi,"American (Traditional), Food, Bakeries, Restau...",1.5
1,kgffcoxT6BQp-gJ-UQ7Czw,Subway,"Fast Food, Restaurants, Sandwiches",2.5
2,0jtRI7hVMpQHpUVtUy4ITw,Omelet House Summerlin,"Beer, Wine & Spirits, Italian, Food, American ...",4.0
3,JJEx5wIqs9iGGATOagE8Sg,Baja Fresh Mexican Grill,"Mexican, Restaurants",2.0
4,zhxnD7J5_sCrKSw5cwI9dQ,Popeyes Louisiana Kitchen,"Chicken Wings, Restaurants, Fast Food",1.5


## Filter review data by "business_id"
* We want to make a DataFrame that contain and only contain the reviews about the business entities we just obtained

** Load review dataset**


In [27]:
df_left = df_selected_business.set_index('business_id')

df_right  = df_review.set_index('business_id')
    
#use inner join to join df_left and df_right
df_joined = df_left.join(df_right, how = 'inner')
df_joined.reset_index(inplace = True)
    

del df_review
del df_right

My computer moving so slow, I will filter data by date, keep comments from last 2 years
* otherwise my laptop may crush on memory when running machine learning algorithm
* purposefully ignoring the reviews made too long time ago

In [30]:
# Make a filter that selects date after 2016-09-01
cond_last_2years = df_joined['date']> u'2016-09-01'

In [32]:
# Filter df_final
df_final = df_joined[cond_last_2years]

In [33]:
df_final.head()

,business_id,name,categories,avg_stars,cool,date,funny,review_id,stars,text,useful,user_id
1,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2017-02-14,0,VETXTwMw6qxzOVDlXfe6Tg,5,went for dinner tonight. Amazing my husband ha...,0,ymlnR8UeFvB4FZL56tCZsA
6,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2017-12-04,0,S8-8uZ7fa5YbjnEtaW15ng,5,This was an amazing dinning experience! ORDER ...,0,9pSSL6X6lFpY3FCRLEH3og
15,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2016-09-13,0,N1Z93BthdJ7FT2p5S22jIA,3,Went for a nice anniversary dinner. Researched...,0,CEtidlXNyQzgJSdF1ubPFw
31,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2017-05-20,0,Pnkrj90xfykhHyo4BSFRsw,5,ABSOLUTE MUST IN VEGAS! Loved everything my bo...,0,cZVQGCZ_fHtTdfiyGVJPdg
45,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2017-12-30,0,Oeh7e6U2xaDQI9L9i4x_Gw,2,I had high hopes for Delmonico's Steakhouse in...,0,li2cBZl60vgqihDJJG7jeA


## Save the preprocessed dataset to csv file

I don't want to run everything again

In [34]:
df_final.to_csv('last_2_years_restaurant_reviews.csv', index= False, encoding='utf8')

In [36]:
# test if it works
pd.read_csv('last_2_years_restaurant_reviews.csv', encoding = 'utf8').head()

,business_id,name,categories,avg_stars,cool,date,funny,review_id,stars,text,useful,user_id
0,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2017-02-14,0,VETXTwMw6qxzOVDlXfe6Tg,5,went for dinner tonight. Amazing my husband ha...,0,ymlnR8UeFvB4FZL56tCZsA
1,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2017-12-04,0,S8-8uZ7fa5YbjnEtaW15ng,5,This was an amazing dinning experience! ORDER ...,0,9pSSL6X6lFpY3FCRLEH3og
2,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2016-09-13,0,N1Z93BthdJ7FT2p5S22jIA,3,Went for a nice anniversary dinner. Researched...,0,CEtidlXNyQzgJSdF1ubPFw
3,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2017-05-20,0,Pnkrj90xfykhHyo4BSFRsw,5,ABSOLUTE MUST IN VEGAS! Loved everything my bo...,0,cZVQGCZ_fHtTdfiyGVJPdg
4,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"Steakhouses, Restaurants, Cajun/Creole",4.0,0,2017-12-30,0,Oeh7e6U2xaDQI9L9i4x_Gw,2,I had high hopes for Delmonico's Steakhouse in...,0,li2cBZl60vgqihDJJG7jeA
